
- Projeto ADA 

Bibliotecas
Eu precisei instalar previamente o "Sopa bonita e o Dotenv"
Sobre o App
Necessário Criar conta no Azure e minha chave.

Conteúdo ajustado do Forum DIO
Link original: https://web.dio.me/topics/projeto-traducao-de-artigos-passo-a-passo-1?back=%2Ftrack%2Fmicrosoft-certification-challenge-4-ai-102&page=1&order=oldest

In [ ]:
# Importando as bibliotecas e variáveis de sistema

from bs4 import BeautifulSoup
import requests, uuid, json
import os
from dotenv import load_dotenv

# Carregar variáveis de ambiente do arquivo .env
load_dotenv()
azureai_endpoint = os.getenv("AZURE_ENDPOINT")
API_KEY = os.getenv("AZURE_OPENAI_KEY")

# Importante definir as variáveis de sistema dentro das variáveis de ambiente para não ter exposta a chave e endpoint.

url="https://www.researchgate.net/publication/352702749_Galinha_Pintadinha_Runs_the_World_A_Made-for-Children_Brazilian_Cartoon_in_the_Global_Flow_of_Television_Content"
def extract_text(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        for script in soup(["script", "style"]):
            script.decompose()
        text = soup.get_text(" ", strip=True)
        return text
    else:
        print("Falha ao buscar a URL. Código de status:", response.status_code)
        return None
extract_text(url)

def traduzir_artigo(text, lang):
    headers = {
    "Content-Type": "application/json",
    "api-key": API_KEY,
    }
    
    # Definindo as instruções do agente e a sua formatação
    payload = {
      "messages": [
        {
          "role": "system",
          "content": [
            {
              "type": "text",
              "text": "Você atua como tradutor de textos"
            }
          ]
        },
        {
          "role": "user",
          "content": [
            {
              "type": "text",
              "text": f"traduza: {text} para o idioma {lang} e responda apenas com a tradução no formato markdown"
            }
          ]
        },    
      ],
      "temperature": 0.7,
      "top_p": 0.95,
      "max_tokens": 900
    }
    
    ENDPOINT = azureai_endpoint
    
    # Enviando a requisição
    try:
        response = requests.post(ENDPOINT, headers=headers, json=payload)
        response.raise_for_status()  
        # Teste de resposta do HTTPError ou falha de requisição
    except requests.RequestException as e:
        raise SystemExit(f"Falha ao enviar request. Erro: {e}")
    try:
      data = response.json()
    except ValueError:
      raise SystemExit("Falha ao decodificar JSON da resposta")

    if not isinstance(data, dict) or 'choices' not in data or not data['choices']:
      raise SystemExit(f"Formato de resposta inesperado: {data}")

    # resposta válida recebida (pode descomentar para debug)
    # print("Resposta recebida com sucesso")
    return (response.json()['choices'][0]['message']['content'])




SyntaxError: invalid syntax (2030830596.py, line 12)

In [ ]:
url = "https://www.researchgate.net/publication/352702749_Galinha_Pintadinha_Runs_the_World_A_Made-for-Children_Brazilian_Cartoon_in_the_Global_Flow_of_Television_Content"
text = extract_text(url)
artigo = traduzir_artigo(text,"português")
print(artigo)